# Modelling LSTM, GRADIENT BOOSTING, FAST TEXT + RFC, FAST TEXT + LSTM


## ImportS

In [2]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import spacy
import ast
import joblib


from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from ast import literal_eval



from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix
from gensim.models import Word2Vec

from sklearn.ensemble import GradientBoostingClassifier

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
#this initialize tqdm which is useful to show a progress bar when applying operations in a pandas df
tqdm.pandas()

## Data Extract

In [2]:
df = pd.read_csv('data/undersampled_data_60_40.csv')


In [5]:
df.columns

Index(['comment_text', 'toxic', 'stopwords_punct_lemma', 'vector_spacy',
       'pos_tags', 'pos_tags_str'],
      dtype='object')

In [6]:
#drop NaNs from df['stopwords_punct_lemma']
df.dropna(subset=['stopwords_punct_lemma'], inplace=True)


## Function to Calculate

In [7]:
# initialize dataframe that will include the results
results_df = pd.DataFrame()

def evaluate_model(model, X_train,y_train,X_test,y_test, model_name="", parameters='', comments=''):
    start_time = time.time()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    duration = time.time() - start_time
    duration_format = f"{int(duration // 60)} minutes and {round(duration % 60, 2)} seconds"
    predicted_probs = model.predict_proba(X_test)[:, 1]

    # Calculate metrics using probabilities
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, predicted_probs)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    conf_matrix = str(confusion_matrix(y_test, predictions))

    # Create a dictionary including the results
    results = {
        'Name': model_name if model_name else model.__class__.__name__,
        'Parameters': parameters,
        'F1-Score': f1,
        'AUC-ROC': roc_auc,
        'Precision': precision,
        'Recall': recall,
        'Accuracy': accuracy,
        'Confusion Matrix': conf_matrix,
        'Training Time': duration_format,
        'Comments': comments
    }

    return results

## Gradient Boosting

In [8]:
# Prepare the data
X = df['stopwords_punct_lemma']
y = df['toxic']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform on the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

In [9]:
# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier()

# Evaluate Gradient Boosting Classifier using evaluate_model function
results_gb = evaluate_model(gb_classifier, X_train_tfidf, y_train, X_test_tfidf, y_test, model_name="Gradient Boosting Classifier", parameters="TF-IDF")

# Convert the dictionary to a DataFrame
results_df_gb = pd.DataFrame([results_gb])

# Concatenate the new results DataFrame to the existing one
results_df = pd.concat([results_df, results_df_gb], ignore_index=True)

# Display the results
print(results_df)


                           Name Parameters  F1-Score   AUC-ROC  Precision  \
0  Gradient Boosting Classifier     TF-IDF  0.659907  0.850829   0.914691   

     Recall  Accuracy                 Confusion Matrix  \
0  0.516138  0.785904  [[41665  1396]\n [14032 14968]]   

                 Training Time Comments  
0  2 minutes and 58.07 seconds           


In [10]:
results_df

,Name,Parameters,F1-Score,AUC-ROC,Precision,Recall,Accuracy,Confusion Matrix,Training Time,Comments
0,Gradient Boosting Classifier,TF-IDF,0.659907,0.850829,0.914691,0.516138,0.785904,[[41665 1396]\n [14032 14968]],2 minutes and 58.07 seconds,


## LSTM

In [6]:
X = df['comment_text'].values
y = df['toxic'].values 

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Tokenize and convert text to sequences
max_words = 10000  # Set the maximum number of words to consider
max_len = 100  # Set the maximum length of each sequence
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [7]:
# Save the tokenizer to a file
tokenizer_file_path = 'data/tokenizer.pkl'
with open(tokenizer_file_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [8]:
# Pad sequences to a fixed length
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(units=64))
model.add(Dense(units=1, activation='sigmoid'))

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_data=(X_test_padded, y_test))



Epoch 1/5
9021/9021 [==============================] - 247s 27ms/step - loss: 0.3341 - accuracy: 0.8596 - val_loss: 0.3001 - val_accuracy: 0.8755
Epoch 2/5
9021/9021 [==============================] - 239s 26ms/step - loss: 0.2868 - accuracy: 0.8808 - val_loss: 0.2987 - val_accuracy: 0.8756
Epoch 3/5
9021/9021 [==============================] - 248s 27ms/step - loss: 0.2627 - accuracy: 0.8915 - val_loss: 0.3014 - val_accuracy: 0.8744
Epoch 4/5
9021/9021 [==============================] - 243s 27ms/step - loss: 0.2357 - accuracy: 0.9026 - val_loss: 0.3127 - val_accuracy: 0.8720
Epoch 5/5
9021/9021 [==============================] - 228s 25ms/step - loss: 0.2035 - accuracy: 0.9166 - val_loss: 0.3454 - val_accuracy: 0.8664


In [76]:
# Save the model architecture as JSON
model_json = model.to_json()
with open('model5.json', 'w') as json_file:
    json_file.write(model_json)

# Save the model weights
model.save_weights('model_weights5.h5')

In [23]:
'''
with open('lstm_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
    '''

In [77]:
# Generate predictions
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict(X_test_padded))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")
print(f"Confusion Matrix:\n{conf_matrix}")

2256/2256 [==============================] - 14s 6ms/step
Accuracy: 0.8389152936937936
Precision: 0.8017638036809815
Recall: 0.7948097722796227
F1 Score: 0.7982716435004424
AUC-ROC: 0.9027370638487501
Confusion Matrix:
[[37541  5687]
 [ 5938 23001]]


## FAST Text

In [30]:
ft_df = pd.read_csv('data/undersampled_data_60_40_ft.csv')


In [11]:
ft_df.columns

Index(['Unnamed: 0', 'comment_text', 'toxic', 'stopwords_punct_lemma',
       'toxic_label_ft', 'toxic_label_comment_text', 'vector_fast_text'],
      dtype='object')

In [50]:
ft_df['new_ft'].head()

0    [-0.0577833019, 0.0458838157, -0.0487854704, -...
1    [-0.0385174714, 0.0294841994, -0.0353648514, -...
2    [0.08621803, -0.06944817, 0.08360571, 0.003052...
3    [-0.02172438, 0.01810819, -0.02264511, -0.0008...
4    [-0.04083619, 0.03226621, -0.03952266, -0.0020...
Name: new_ft, dtype: object

In [33]:
corpus_vect = ft_df['vector_fast_text'].str.strip('[]').str.split(expand=True)
corpus_vect = corpus_vect.astype('float')

In [34]:
ft_df['new_ft'] = corpus_vect.values.tolist()


### RFC + fast_text

In [37]:
X = np.array(ft_df['new_ft'].tolist())
y = ft_df['toxic']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Flatten X_train and X_test
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)

# Initialize RandomForestClassifier
rfc = RandomForestClassifier()

# Fit the model
rfc.fit(X_train_flatten, y_train)

# Predict on the test set
y_pred = rfc.predict(X_test_flatten)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9696090811950986


In [38]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

# Calculate AUC-ROC
auc_roc = roc_auc_score(y_test, rfc.predict_proba(X_test_flatten)[:, 1])
print(f"AUC-ROC: {auc_roc}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.9696090811950986
Precision: 0.9666179337231969
Recall: 0.957551724137931
F1 Score: 0.9620634700665189
AUC-ROC: 0.9888729088406262
Confusion Matrix:
[[42102   959]
 [ 1231 27769]]


### LSTM + fasttext

In [51]:
X = np.array(ft_df['new_ft'].tolist())
y = ft_df['toxic']

In [58]:
max_words = 10000  # Set the maximum number of words to consider
max_len = 100  # Set the maximum length of each sequence

X_padded = pad_sequences(X, maxlen=max_len) 

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


# Define the LSTM model
model = Sequential()
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

model.add(LSTM(units=64, input_shape=(X_train.shape[1], 1))) 
model.add(Dense(units=1, activation='sigmoid'))



In [60]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
9008/9008 [==============================] - 179s 20ms/step - loss: 0.6713 - accuracy: 0.6022 - val_loss: 0.6714 - val_accuracy: 0.6005
Epoch 2/5
9008/9008 [==============================] - 172s 19ms/step - loss: 0.6705 - accuracy: 0.6028 - val_loss: 0.6715 - val_accuracy: 0.6005
Epoch 3/5
9008/9008 [==============================] - 173s 19ms/step - loss: 0.6705 - accuracy: 0.6028 - val_loss: 0.6714 - val_accuracy: 0.6005
Epoch 4/5
9008/9008 [==============================] - 174s 19ms/step - loss: 0.6720 - accuracy: 0.6018 - val_loss: 0.6715 - val_accuracy: 0.6004
Epoch 5/5
9008/9008 [==============================] - 178s 20ms/step - loss: 0.6708 - accuracy: 0.6027 - val_loss: 0.6714 - val_accuracy: 0.6004


In [61]:
# save the model in .h5 file
model.save('lstm_model_ft.h5')

In [63]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict(X_test))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")
print(f"Confusion Matrix:\n{conf_matrix}")

2252/2252 [==============================] - 14s 6ms/step
Accuracy: 0.6004079876771069
Precision: 0.9723502304147466
Recall: 0.0072758620689655175
F1 Score: 0.014443645822637506
AUC-ROC: 0.5034765537100937
Confusion Matrix:
[[43055     6]
 [28789   211]]


### LSTM on cleaned

In [3]:
df_clean = pd.read_csv('data/data_usampl_60_40_comments_cleaned_preproc.csv')


In [9]:
df_clean.head()

,comment_clean,comment_clean_preproc,toxic
0,"Well, what are the chances he will turn out to...",chance turn active proponent slavery,0
1,The moment of critical mass is approaching whe...,moment critical mass approach deed Gupta Co li...,0
2,"""Hey listen to me,"" he said. ""I'm not going to...",hey listen say go crap prove reporter say uh a...,1
3,We are already owed $_number_ M plus interest(...,owe $ number M plus interest($_number_Billion ...,0
4,There is a reason there are no teeth to the la...,reason tooth law unlawful law way force free e...,0


In [13]:
df_clean['comment_clean'] = df_clean['comment_clean'].astype(str)
df_clean['comment_clean_preproc'] = df_clean['comment_clean_preproc'].astype(str)


In [14]:
X = df_clean['comment_clean_preproc'].values
y = df_clean['toxic'].values 

In [15]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Tokenize and convert text to sequences
max_words = 10000  # Set the maximum number of words to consider
max_len = 100  # Set the maximum length of each sequence
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [16]:
# Save the tokenizer to a file
tokenizer_file_path = 'data/tokenizer_clean.pkl'
with open(tokenizer_file_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
# Pad sequences to a fixed length
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)

In [18]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(units=64))
model.add(Dense(units=1, activation='sigmoid'))

In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_data=(X_test_padded, y_test))


Epoch 1/5


2023-12-13 15:41:11.776438: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


9021/9021 [==============================] - 236s 26ms/step - loss: 0.3055 - accuracy: 0.8713 - val_loss: 0.2822 - val_accuracy: 0.8827
Epoch 2/5
9021/9021 [==============================] - 231s 26ms/step - loss: 0.2672 - accuracy: 0.8879 - val_loss: 0.2812 - val_accuracy: 0.8821
Epoch 3/5
9021/9021 [==============================] - 235s 26ms/step - loss: 0.2406 - accuracy: 0.9000 - val_loss: 0.2916 - val_accuracy: 0.8809
Epoch 4/5
9021/9021 [==============================] - 226s 25ms/step - loss: 0.2099 - accuracy: 0.9134 - val_loss: 0.3164 - val_accuracy: 0.8736
Epoch 5/5
9021/9021 [==============================] - 232s 26ms/step - loss: 0.1774 - accuracy: 0.9278 - val_loss: 0.3414 - val_accuracy: 0.8666


In [20]:
# Save the model architecture as JSON
model_json = model.to_json()
with open('data/model_clean.json', 'w') as json_file:
    json_file.write(model_json)

# Save the model weights
model.save_weights('data/model_weights_clean.h5')


In [21]:
# Generate predictions
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict(X_test_padded))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")
print(f"Confusion Matrix:\n{conf_matrix}")


2256/2256 [==============================] - 15s 7ms/step
Accuracy: 0.8665595078082781
Precision: 0.8288205442593917
Recall: 0.8409067348560766
F1 Score: 0.834819897084048
AUC-ROC: 0.9346512256536599
Confusion Matrix:
[[38202  5026]
 [ 4604 24335]]
